# LLM application with LangChain and Cohere

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN API KEY:")

LANGCHAIN API KEY: ········


In [2]:

os.environ["COHERE_API_KEY"] = getpass.getpass("COHERE API KEY:")

COHERE API KEY: ········


In [3]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '92fe3771-a462-4359-a605-63977822ac07', 'token_count': {'input_tokens': 78, 'output_tokens': 3}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '92fe3771-a462-4359-a605-63977822ac07', 'token_count': {'input_tokens': 78, 'output_tokens': 3}}, id='run-e331bea7-bcf8-4eff-96b2-3524a2ad933a-0', usage_metadata={'input_tokens': 78, 'output_tokens': 3, 'total_tokens': 81})

We parse out this response by using a simple output parser:

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [6]:
result = model.invoke(messages)

In [7]:
parser.invoke(result)

'Ciao!'

A resposta do modelo é do tipo `string` e contem várias informações além da tradução. Podemos "encadear" o modelo com "output parser".

In [8]:
# Criamos a cadeia através do operador |

chain = model | parser

In [9]:
chain.invoke(messages)

'Ciao!'

Vamos criar um `PromptTemplate` com duas variáveis:
 * `language`: a língua para qual queremos traduzir
 * `text`: o texto para ser traduzido

In [10]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:" # criamos system message

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [11]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [12]:
# criamos a cadeia
chain = prompt_template | model | parser

In [13]:
# chamamos
chain.invoke({"language": "italian", "text": "hi"})

'Ciao! In che modo posso aiutarti oggi?'

## Servimos o app com LangServe

Criamos o arquivo `serve.py` que vai criar um servidor para o app. O arquivo contem:

    A definição da cadeia que construimos acima
    Nosso app de FastAPI 
    A definição definição de uma rota a partir da qual servir a cadeia, que é feita com `langserve.add_routes`

In [17]:
from langchain_openai import ChatOpenAI